## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Nikolskoye,RU,59.70,30.79,62.01,87,20,4.47,few clouds
1,1,Barrow,US,71.29,-156.79,42.80,81,90,9.17,overcast clouds
2,2,Masallatah,LY,32.62,14.00,76.73,84,60,11.30,broken clouds
3,3,Lodja,CD,-3.48,23.43,69.76,91,100,2.59,overcast clouds
4,4,Sao Joao Da Barra,BR,-21.64,-41.05,64.89,72,3,11.61,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Masallatah,LY,32.62,14.00,76.73,84,60,11.30,broken clouds
7,7,Drumheller,CA,51.46,-112.71,78.01,36,61,6.51,broken clouds
9,9,Bubaque,GW,11.28,-15.83,80.71,82,98,10.18,overcast clouds
12,12,Wanning,CN,18.80,110.40,81.84,82,99,8.68,overcast clouds
18,18,Kalianget,ID,-7.05,113.93,81.30,71,19,14.07,few clouds
26,26,Isangel,VU,-19.55,169.27,80.60,94,75,4.81,broken clouds
27,27,Lazaro Cardenas,MX,17.96,-102.20,82.80,83,34,5.14,light rain
31,31,Ponta Do Sol,PT,32.67,-17.10,75.20,64,20,16.11,few clouds
36,36,Kavieng,PG,-2.57,150.80,76.78,92,100,10.78,heavy intensity rain
38,38,Bandiagara,ML,14.35,-3.61,80.60,65,21,8.25,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        256
City           256
Country        256
Lat            256
Lng            256
Max Temp       256
Humidity       256
Cloudiness     256
Wind Speed     256
Description    256
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Masallatah,LY,76.73,broken clouds,32.62,14.00,
7,Drumheller,CA,78.01,broken clouds,51.46,-112.71,
9,Bubaque,GW,80.71,overcast clouds,11.28,-15.83,
12,Wanning,CN,81.84,overcast clouds,18.80,110.40,
18,Kalianget,ID,81.30,few clouds,-7.05,113.93,
26,Isangel,VU,80.60,broken clouds,-19.55,169.27,
27,Lazaro Cardenas,MX,82.80,light rain,17.96,-102.20,
31,Ponta Do Sol,PT,75.20,few clouds,32.67,-17.10,
36,Kavieng,PG,76.78,heavy intensity rain,-2.57,150.80,
38,Bandiagara,ML,80.60,few clouds,14.35,-3.61,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.drop(index, inplace=True)

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))